In [16]:
import tensorflow as tf

#하이퍼 파라미터 설정부

In [17]:
epochs = 5

#네트워크 구조 정의


In [18]:
class MyModel(tf.keras.Model):
  def __init__(self):
    #레이어를 정의하는곳
    super(MyModel,self).__init__()
    self.flatten = tf.keras.layers.Flatten(input_shape=(28,28))
    self.d1 = tf.keras.layers.Dense(128,activation='sigmoid')
    self.d2 = tf.keras.layers.Dense(10,activation='softmax')

  def call(self,x):
    #호출시 어떻게 연결이 되어있는지
    x=self.flatten(x)
    x=self.d1(x)
    return self.d2(x)

#학습 루프 정의

In [19]:
@tf.function
def train_step(model,images,labels,loss_object,optimizer,train_loss,train_accuracy):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)

  optimizer.apply_gradients(zip(gradients,model.trainable_variables))
  train_loss(loss)
  train_accuracy(labels,predictions)

#테스트 루프 정의

In [20]:
@tf.function
def test_step(model,images,labels,loss_object,test_loss,test_accuracy):
  predictions = model(images)

  t_loss = loss_object(labels,predictions)
  test_loss(t_loss)
  test_accuracy(labels,predictions)

#정의는 끝났고, 데이터셋 가져오고, 정리

In [21]:
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test,y_test) = mnist.load_data()
x_train,x_test = x_train/255.0,x_test/255.0

x_train = x_train[...,tf.newaxis]
x_test = x_test[...,tf.newaxis]

train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)

#모델을 생성

In [22]:
model = MyModel()

#loss func이랑 optimizer정의

In [23]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

#알고리즘 평가지표 설정

In [27]:
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

#학습진행

In [29]:
for epoch in range(epochs):
  for images,labels in train_ds:
    train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

  for test_images, test_labels in test_ds:
    test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

  template = 'Epoch:{}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

Epoch:1, Loss: 0.011992149986326694, Accuracy: 99.83333587646484, Test Loss: 0.07737460732460022, Test Accuracy: 97.79999542236328
Epoch:2, Loss: 0.01014743186533451, Accuracy: 99.8616714477539, Test Loss: 0.07829583436250687, Test Accuracy: 97.55999755859375
Epoch:3, Loss: 0.008063416928052902, Accuracy: 99.92166900634766, Test Loss: 0.07565972208976746, Test Accuracy: 97.79000091552734
Epoch:4, Loss: 0.006834546569734812, Accuracy: 99.93167114257812, Test Loss: 0.078769251704216, Test Accuracy: 97.66999816894531
Epoch:5, Loss: 0.005586069077253342, Accuracy: 99.94499969482422, Test Loss: 0.08111285418272018, Test Accuracy: 97.70999908447266
